## **Project 2**

Directory ID: swarajmr
Name: Swaraj Mundruppady Rao
UID: 120127007

Problem 1 Solution:

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
#Importing Dependent Libraries
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import math

In [39]:
#Define the path to the video file
cap=cv.VideoCapture("/content/drive/MyDrive/ENPM673/Project2/proj2_v2.mp4")

#Get the frame width, height and frames per second of the video
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

#Decrease the frames per second to 10 to get a better view of the edges and corners
fps = 10

# Define the codec and create VideoWriter object
out = cv.VideoWriter('/content/drive/MyDrive/ENPM673/Project2/swarajmr_project2_p1.mp4',
                      cv.VideoWriter_fourcc(*'MP4V'), fps, (frame_width, frame_height))

#Variables to keep track of the number of frames skipped and total number of frames
pointmatching=0
totalframes=0
skippedframes=0

#Threshold for the minimum line length to be considered as a valid line
min_line_length_threshold = 131.8

#Loop through the video frames
while cap.isOpened():
  ret,frame=cap.read()
  #Count Total Number of Frames
  totalframes+=1
  if not ret:
    print("Can't receive frame(stream end? ). Exiting...")
    break

  #Converting image to gray to detect the edges
  gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

  #Filtering out the blurry frames from the video, using Threshold of 118
  laplaciankernel = np.array([[0,1,0],[1,-4,1],[0,1,0]], dtype=np.float32)

  #Finding the variance of the frame
  var=cv.filter2D(frame, cv.CV_64F, kernel=laplaciankernel).var()

  #Skipping the blurry frames and writing the non-blurry frames to the output video
  if var<118:
    skippedframes+=1
    continue

  #Using the non-blurry frames to detect the edges of the paper and draw lines
  else:
    #Gaussian Blurr to detect the edges
    blurred = cv.GaussianBlur(gray, (5,5),0)
    #Converting the frame to a binary image with a threshold range of 200 - 255
    _, binary = cv.threshold(blurred, 200, 255, cv.THRESH_BINARY)
    #Canny Edge Detector
    edges = cv.Canny(binary, 50, 150)
    #Computing the lines using Probabilistic Hough Lines Transform
    lines = cv.HoughLinesP(edges, rho=1, theta=1*np.pi/180, threshold=65, minLineLength=15, maxLineGap=10)
    #Harris Corner Detector
    dst = cv.cornerHarris(gray, 2, 3, 0.04)
    dst = cv.dilate(dst, None)
    threshold = 0.7 * dst.max()
    corner_coordinates = np.argwhere(dst > threshold)
    cartesian_lines = []

    #Detecting the end points of the paper using intersection of lines
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cartesian_lines.append(((x1, y1), (x2, y2)))
    # To find intersections
    intersections = []
    for i in range(len(cartesian_lines)):
      for j in range(i + 1, len(cartesian_lines)):
        line1 = cartesian_lines[i]
        line2 = cartesian_lines[j]
        x1, y1 = line1[0]
        x2, y2 = line1[1]
        x3, y3 = line2[0]
        x4, y4 = line2[1]
        linelength1=np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        linelength2=np.sqrt((x4 - x3)**2 + (y4 - y3)**2)

        # Calculating slopes of the lines
        slope1 = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else math.inf
        slope2 = (y4 - y3) / (x4 - x3) if (x4 - x3) != 0 else math.inf

        # Calculate angle between the lines
        angle = abs(math.atan((slope2 - slope1) / (1 + slope1 * slope2)))
        angle_threshold = math.radians(20)
        #mark only points for nearly Perpendicular lines
        if angle > angle_threshold:
          # Calculate intersection point of the lines
          denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
          if denom != 0 and linelength1>min_line_length_threshold and linelength2>min_line_length_threshold:
            x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
            y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom
            intersections.append((int(x), int(y)))


    #Marking the intersection points
    for point in intersections:
      cv.circle(frame, point, 5, (0, 0, 255), -1)

    # Drawing the lines on the image
    for line in cartesian_lines:
      x1,y1=line[0]
      x2,y2=line[1]
      line_length=np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
      if line_length > min_line_length_threshold:
        cv.line(frame, line[0], line[1], (255, 0, 0), 3)

    #Corner verification using Harris Corner Co-ordinates( The points will be marked in green )
    for coord_harris in corner_coordinates:
      for point in intersections:
        if np.linalg.norm(np.array(point) - np.array(coord_harris[::-1])) < 15:
            cv.circle(frame, point, 5, (0, 0, 255), 1)
    out.write(frame)

cap.release()
out.release()
cv.destroyAllWindows()

Can't receive frame(stream end? ). Exiting...


In [40]:
#Total Number of Frames Skipped
print("Number of Frames Skipped : ",skippedframes)
print("Total Frames: ",totalframes)
print("Number of frames in the video output: ",(totalframes-skippedframes))
print((totalframes-skippedframes)/totalframes)

Number of Frames Skipped :  193
Total Frames:  387
Number of frames in the video output:  194
0.5012919896640827
